In [1]:
import pandas as pd #to work with data frames aka the holy grail
import numpy as np #useful library to help us manipulate data frames 
import json #read and write json files 
import requests #make spotify api call requests
from pandas.io.json import json_normalize
from io import StringIO
from urllib.parse import urlencode
import base64



In [2]:
#reading streaming data into data frames 
df_stream0 = pd.read_json("StreamingHistory0.json")

In [3]:
df_stream1 = pd.read_json("StreamingHistory1.json")

In [4]:
df_stream2 = pd.read_json("StreamingHistory2.json")

In [5]:
df_stream = pd.concat([df_stream0, df_stream1, df_stream2])


In [6]:
#what does our streaming history data frame tell us 
df_stream.head()

,endTime,artistName,trackName,msPlayed
0,2021-01-09 01:21,Playboi Carti,JumpOutTheHouse,132800
1,2021-01-09 01:26,Playboi Carti,JumpOutTheHouse,325432
2,2021-01-09 01:29,Playboi Carti,M3tamorphosis (feat. Kid Cudi),150021
3,2021-01-09 01:41,Playboi Carti,JumpOutTheHouse,136297
4,2021-01-09 01:45,Playboi Carti,JumpOutTheHouse,16171


In [7]:
#lets add a uniqueID for each row
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

In [8]:
df_stream.head()
#yea... I was a huge fan of whole lotta red it seems lol 


,endTime,artistName,trackName,msPlayed,UniqueID
0,2021-01-09 01:21,Playboi Carti,JumpOutTheHouse,132800,Playboi Carti:JumpOutTheHouse
1,2021-01-09 01:26,Playboi Carti,JumpOutTheHouse,325432,Playboi Carti:JumpOutTheHouse
2,2021-01-09 01:29,Playboi Carti,M3tamorphosis (feat. Kid Cudi),150021,Playboi Carti:M3tamorphosis (feat. Kid Cudi)
3,2021-01-09 01:41,Playboi Carti,JumpOutTheHouse,136297,Playboi Carti:JumpOutTheHouse
4,2021-01-09 01:45,Playboi Carti,JumpOutTheHouse,16171,Playboi Carti:JumpOutTheHouse


# isolate the "tracks" dictionary from the library file and save it as a new file 


In [9]:
#let's attempt to read the json file regularly 


In [10]:
#open the library json file and save it
with open("YourLibrary.json", "r",encoding="utf8") as read_file:
    lib = json.load(read_file)

In [11]:
#this a dictionary
type(lib)

dict

In [12]:
#lets see what it is in it
lib.keys()

dict_keys(['tracks', 'albums', 'shows', 'episodes', 'bannedTracks', 'artists', 'bannedArtists', 'other'])

In [13]:
#we only care about the tracks for this workshop we will isolate this

In [14]:
tracks = lib['tracks']
#this returns a list 
print(tracks[0])

{'artist': 'Radiant Children', 'album': "Life's a Bitch", 'track': "Life's a Bitch", 'uri': 'spotify:track:2kMflL8jH2U1UaEhS6W25z'}


In [15]:
help(requests)

Help on package requests:

NAME
    requests

DESCRIPTION
    Requests HTTP Library
    ~~~~~~~~~~~~~~~~~~~~~
    
    Requests is an HTTP library, written in Python, for human beings.
    Basic GET usage:
    
       >>> import requests
       >>> r = requests.get('https://www.python.org')
       >>> r.status_code
       200
       >>> b'Python is a programming language' in r.content
       True
    
    ... or POST:
    
       >>> payload = dict(key1='value1', key2='value2')
       >>> r = requests.post('https://httpbin.org/post', data=payload)
       >>> print(r.text)
       {
         ...
         "form": {
           "key1": "value1",
           "key2": "value2"
         },
         ...
       }
    
    The other HTTP methods are supported - see `requests.api`. Full documentation
    is at <https://requests.readthedocs.io>.
    
    :copyright: (c) 2017 by Kenneth Reitz.
    :license: Apache 2.0, see LICENSE for more details.

PACKAGE CONTENTS
    __version__
    _internal_utils

In [16]:
#Accessing the Spotify API
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '54fea533c6314e99bf718911b800f564'
CLIENT_SECRET = "27b01cb9cc0a47f9b4ada883841eab6e"

In [17]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'



In [18]:
help(requests.post)

Help on function post in module requests.api:

post(url, data=None, json=None, **kwargs)
    Sends a POST request.
    
    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) json data to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [19]:
# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})



In [87]:
auth_response_data = auth_response.json()

In [88]:
auth_response_data.keys()

dict_keys(['access_token', 'token_type', 'expires_in'])

In [89]:
access_token = auth_response_data['access_token']
print(auth_response.json())

{'access_token': 'BQDdBDBUmvz2NnuqU1KMMHh73VCWsXM3l7av4H4ZINYAprNYqEMTYJs1pHc-LcvaSwDCiZDiRQTbDOAH7po', 'token_type': 'Bearer', 'expires_in': 3600}


In [96]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}
BASE_URL = 'https://api.spotify.com/v1/'


In [97]:
#this a test to get a song from the api based on the track uri

In [98]:
type(tracks[0])
#each index in the list is a dictionary

dict

In [99]:
print(tracks[0])

{'artist': 'Radiant Children', 'album': "Life's a Bitch", 'track': "Life's a Bitch", 'uri': 'spotify:track:2kMflL8jH2U1UaEhS6W25z'}


In [100]:
track = tracks[0]
uri = track['uri']
uri

'spotify:track:2kMflL8jH2U1UaEhS6W25z'

In [101]:
type(uri)

str

In [102]:
uri = uri.split(":")

In [103]:
t_uri = uri[2]

In [104]:
r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)


In [105]:
type(r)

requests.models.Response

In [106]:
r = r.json()
type(r)

dict

In [107]:
r.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [112]:
artist_uri = r['artists'][0]['uri']

In [113]:
artist_uri = artist_uri.split(":")

In [114]:
a_uri = artist_uri[2]

In [115]:
s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)


In [116]:
s = s.json()

In [117]:
s.keys()

dict_keys(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name', 'popularity', 'type', 'uri'])

In [118]:
s['genres']

['alternative r&b', 'indie r&b', 'uk contemporary r&b']

In [77]:
df_library =pd.DataFrame(tracks)

In [79]:
df_library.head()

,artist,album,track,uri
0,Radiant Children,Life's a Bitch,Life's a Bitch,spotify:track:2kMflL8jH2U1UaEhS6W25z
1,Lil Mosey,G Walk (with Chris Brown),G Walk (with Chris Brown),spotify:track:4E08gEca6avoGx5SS4MM6T
2,SpotemGottem,Beat Box 3 (feat. DaBaby),Beat Box 3 (feat. DaBaby),spotify:track:6toQdWWc4noiOk3Eo5mVDS
3,Peezy,Rio Flow,Rio Flow,spotify:track:4MdraTedPRjTzFhXKPvnfb
4,Migos,Culture III,Why Not,spotify:track:1S1dOc3yIPNt4sNxhlOghf


# We downloaded the library file as a dictionary
# 2. the Dictionary had keys that represent each album, track, and uri in the dictionary
# 3. We are only concerned with the tracks in our library so we get this key and its value from the dictionary
# 4 when we do we get a list and each index is a dictionary 


In [80]:
df_library.columns
#we've now created an dataframe where we have each song in our library and the artist, album, track, and uri that corresponds

Index(['artist', 'album', 'track', 'uri'], dtype='object')

In [81]:
df_library.head()

,artist,album,track,uri
0,Radiant Children,Life's a Bitch,Life's a Bitch,spotify:track:2kMflL8jH2U1UaEhS6W25z
1,Lil Mosey,G Walk (with Chris Brown),G Walk (with Chris Brown),spotify:track:4E08gEca6avoGx5SS4MM6T
2,SpotemGottem,Beat Box 3 (feat. DaBaby),Beat Box 3 (feat. DaBaby),spotify:track:6toQdWWc4noiOk3Eo5mVDS
3,Peezy,Rio Flow,Rio Flow,spotify:track:4MdraTedPRjTzFhXKPvnfb
4,Migos,Culture III,Why Not,spotify:track:1S1dOc3yIPNt4sNxhlOghf


In [82]:
# we need to create a unique idea for each song. this will come in handy later
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']



In [83]:
#we need the uris for each track. this will also come in handy later.
df_library.loc[:,"uri"]
#notice that the uri is the alphanumerical sequence after the second colon in each row. we need to strip this 

0       spotify:track:2kMflL8jH2U1UaEhS6W25z
1       spotify:track:4E08gEca6avoGx5SS4MM6T
2       spotify:track:6toQdWWc4noiOk3Eo5mVDS
3       spotify:track:4MdraTedPRjTzFhXKPvnfb
4       spotify:track:1S1dOc3yIPNt4sNxhlOghf
                        ...                 
1009    spotify:track:6h4c2iDPVb8GGC6pDnVIIV
1010    spotify:track:6J9FgTr3z44Bw6ABeVL415
1011    spotify:track:0qQZ4lxmzitLd6w5WQXrKE
1012    spotify:track:38iBrrbbXyWaSPkmuDNgjZ
1013    spotify:track:5tsI3xxDHDgzHhn30LTQNz
Name: uri, Length: 1014, dtype: object

In [84]:
# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
new


,0,1,2
0,spotify,track,2kMflL8jH2U1UaEhS6W25z
1,spotify,track,4E08gEca6avoGx5SS4MM6T
2,spotify,track,6toQdWWc4noiOk3Eo5mVDS
3,spotify,track,4MdraTedPRjTzFhXKPvnfb
4,spotify,track,1S1dOc3yIPNt4sNxhlOghf
...,...,...,...
1009,spotify,track,6h4c2iDPVb8GGC6pDnVIIV
1010,spotify,track,6J9FgTr3z44Bw6ABeVL415
1011,spotify,track,0qQZ4lxmzitLd6w5WQXrKE
1012,spotify,track,38iBrrbbXyWaSPkmuDNgjZ


In [85]:
df_library['track_uri'] = new[2]

In [86]:
df_library.head()
#we now have the trip uri 

,artist,album,track,uri,UniqueID,track_uri
0,Radiant Children,Life's a Bitch,Life's a Bitch,spotify:track:2kMflL8jH2U1UaEhS6W25z,Radiant Children:Life's a Bitch,2kMflL8jH2U1UaEhS6W25z
1,Lil Mosey,G Walk (with Chris Brown),G Walk (with Chris Brown),spotify:track:4E08gEca6avoGx5SS4MM6T,Lil Mosey:G Walk (with Chris Brown),4E08gEca6avoGx5SS4MM6T
2,SpotemGottem,Beat Box 3 (feat. DaBaby),Beat Box 3 (feat. DaBaby),spotify:track:6toQdWWc4noiOk3Eo5mVDS,SpotemGottem:Beat Box 3 (feat. DaBaby),6toQdWWc4noiOk3Eo5mVDS
3,Peezy,Rio Flow,Rio Flow,spotify:track:4MdraTedPRjTzFhXKPvnfb,Peezy:Rio Flow,4MdraTedPRjTzFhXKPvnfb
4,Migos,Culture III,Why Not,spotify:track:1S1dOc3yIPNt4sNxhlOghf,Migos:Why Not,1S1dOc3yIPNt4sNxhlOghf


In [87]:
#Ok so we want to get an idea of tracks that are in our streaming history are in our library.
#the streaming history is an actual recollection of what we listened to over time 
#the library data is the songs we have liked
#there may be a case that certain songs we listen to frequently may not be liked
#let's categorize this in our data frame 


In [88]:
#lets create a data frame that will be used for visualization in tableau
df_tableau = df_stream.copy()



In [89]:
df_tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2021-01-09 01:21,Playboi Carti,JumpOutTheHouse,132800,Playboi Carti:JumpOutTheHouse
1,2021-01-09 01:26,Playboi Carti,JumpOutTheHouse,325432,Playboi Carti:JumpOutTheHouse
2,2021-01-09 01:29,Playboi Carti,M3tamorphosis (feat. Kid Cudi),150021,Playboi Carti:M3tamorphosis (feat. Kid Cudi)
3,2021-01-09 01:41,Playboi Carti,JumpOutTheHouse,136297,Playboi Carti:JumpOutTheHouse
4,2021-01-09 01:45,Playboi Carti,JumpOutTheHouse,16171,Playboi Carti:JumpOutTheHouse


In [90]:
#we'll create a new column that categorizes whether a song in our streaming data is a song in our library
#we can do this by seeing if the uniqueIDs exist in both
#told you those unique ideas would come in handy :)
#df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)



In [91]:
# we now want to merge our streaming data DataFrame and our Library data frame 


In [92]:
# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID,album,track_uri
0,2021-01-09 01:21,Playboi Carti,JumpOutTheHouse,132800,Playboi Carti:JumpOutTheHouse,NaN,NaN
1,2021-01-09 01:26,Playboi Carti,JumpOutTheHouse,325432,Playboi Carti:JumpOutTheHouse,NaN,NaN
2,2021-01-09 01:29,Playboi Carti,M3tamorphosis (feat. Kid Cudi),150021,Playboi Carti:M3tamorphosis (feat. Kid Cudi),Whole Lotta Red,7zLMYtNJcabv4h4wBnjNQI
3,2021-01-09 01:41,Playboi Carti,JumpOutTheHouse,136297,Playboi Carti:JumpOutTheHouse,NaN,NaN
4,2021-01-09 01:45,Playboi Carti,JumpOutTheHouse,16171,Playboi Carti:JumpOutTheHouse,NaN,NaN


In [93]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()


In [94]:
# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    #the key of the dictionary will be the track_uri and the values for each key will be the artist uri and a list of genres
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    #request to get the track information based on the spotify endpoint and the track_uri
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    #turn into json format to access important information 
    r = r.json()
    #get the artist uri from the json file 
    a_uri = r['artists'][0]['uri'].split(':')[2]
    #place the artist uri for the corresponding track uri in the dictionary 
    dict_genre[t_uri]['artist_uri'] = a_uri
    #now use the artist uri to get the genres
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    #turn into json format to access important information 
    s = s.json()
    #put the corresponding genres in the dictionary for th
    dict_genre[t_uri]['genres'] = s['genres']

In [97]:
dict_genre

{'2kMflL8jH2U1UaEhS6W25z': {'artist_uri': '2e0Bkreg0zzfE5umvfKQXf',
  'genres': ['alternative r&b', 'indie r&b', 'uk contemporary r&b']},
 '4E08gEca6avoGx5SS4MM6T': {'artist_uri': '5zctI4wO9XSKS8XwcnqEHk',
  'genres': ['melodic rap', 'rap', 'rap conscient', 'trap', 'vapor trap']},
 '6toQdWWc4noiOk3Eo5mVDS': {'artist_uri': '3Y2MLcrD3ZQXRXmtn4fZQx',
  'genres': ['florida rap', 'viral rap']},
 '4MdraTedPRjTzFhXKPvnfb': {'artist_uri': '6lOaHNmL3l3hKDGNf0egfs',
  'genres': ['detroit trap']},
 '1S1dOc3yIPNt4sNxhlOghf': {'artist_uri': '6oMuImdp5ZcFhWP0ESe6mG',
  'genres': ['atl hip hop', 'hip hop', 'pop rap', 'rap', 'trap']},
 '4S2uhQE8L9V6p7rj7SiauJ': {'artist_uri': '1cNDP5yjU5vjeR8qMf4grg',
  'genres': ['melodic rap', 'trap']},
 '6IAgarq2wDeYzpsY2Kx0Hw': {'artist_uri': '2auiVi8sUZo17dLy1HwrTU',
  'genres': ['afrofuturism',
   'alternative r&b',
   'dance pop',
   'escape room',
   'hip pop',
   'indie soul',
   'neo soul',
   'pop',
   'r&b',
   'urban contemporary']},
 '0SsTkfOhM7e87trwjEq

In [100]:
#convert the dictionary to a Data Frame 
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.head()


,artist_uri,genres
2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,"[alternative r&b, indie r&b, uk contemporary r&b]"
4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,"[melodic rap, rap, rap conscient, trap, vapor ..."
6toQdWWc4noiOk3Eo5mVDS,3Y2MLcrD3ZQXRXmtn4fZQx,"[florida rap, viral rap]"
4MdraTedPRjTzFhXKPvnfb,6lOaHNmL3l3hKDGNf0egfs,[detroit trap]
1S1dOc3yIPNt4sNxhlOghf,6oMuImdp5ZcFhWP0ESe6mG,"[atl hip hop, hip hop, pop rap, rap, trap]"


In [101]:
#make the first column be the track uri
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.head()


,track_uri,artist_uri,genres
2kMflL8jH2U1UaEhS6W25z,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,"[alternative r&b, indie r&b, uk contemporary r&b]"
4E08gEca6avoGx5SS4MM6T,4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,"[melodic rap, rap, rap conscient, trap, vapor ..."
6toQdWWc4noiOk3Eo5mVDS,6toQdWWc4noiOk3Eo5mVDS,3Y2MLcrD3ZQXRXmtn4fZQx,"[florida rap, viral rap]"
4MdraTedPRjTzFhXKPvnfb,4MdraTedPRjTzFhXKPvnfb,6lOaHNmL3l3hKDGNf0egfs,[detroit trap]
1S1dOc3yIPNt4sNxhlOghf,1S1dOc3yIPNt4sNxhlOghf,6oMuImdp5ZcFhWP0ESe6mG,"[atl hip hop, hip hop, pop rap, rap, trap]"


In [102]:
df_genre.reset_index(inplace=True, drop=True)
df_genre.head()



,track_uri,artist_uri,genres
0,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,"[alternative r&b, indie r&b, uk contemporary r&b]"
1,4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,"[melodic rap, rap, rap conscient, trap, vapor ..."
2,6toQdWWc4noiOk3Eo5mVDS,3Y2MLcrD3ZQXRXmtn4fZQx,"[florida rap, viral rap]"
3,4MdraTedPRjTzFhXKPvnfb,6lOaHNmL3l3hKDGNf0egfs,[detroit trap]
4,1S1dOc3yIPNt4sNxhlOghf,6oMuImdp5ZcFhWP0ESe6mG,"[atl hip hop, hip hop, pop rap, rap, trap]"


In [103]:

df_genre.head()

,track_uri,artist_uri,genres
0,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,"[alternative r&b, indie r&b, uk contemporary r&b]"
1,4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,"[melodic rap, rap, rap conscient, trap, vapor ..."
2,6toQdWWc4noiOk3Eo5mVDS,3Y2MLcrD3ZQXRXmtn4fZQx,"[florida rap, viral rap]"
3,4MdraTedPRjTzFhXKPvnfb,6lOaHNmL3l3hKDGNf0egfs,[detroit trap]
4,1S1dOc3yIPNt4sNxhlOghf,6oMuImdp5ZcFhWP0ESe6mG,"[atl hip hop, hip hop, pop rap, rap, trap]"


In [104]:
len(df_genre)

1014

In [105]:
df_tableau

,endTime,artistName,trackName,msPlayed,UniqueID,album,track_uri
0,2021-01-09 01:21,Playboi Carti,JumpOutTheHouse,132800,Playboi Carti:JumpOutTheHouse,NaN,NaN
1,2021-01-09 01:26,Playboi Carti,JumpOutTheHouse,325432,Playboi Carti:JumpOutTheHouse,NaN,NaN
2,2021-01-09 01:29,Playboi Carti,M3tamorphosis (feat. Kid Cudi),150021,Playboi Carti:M3tamorphosis (feat. Kid Cudi),Whole Lotta Red,7zLMYtNJcabv4h4wBnjNQI
3,2021-01-09 01:41,Playboi Carti,JumpOutTheHouse,136297,Playboi Carti:JumpOutTheHouse,NaN,NaN
4,2021-01-09 01:45,Playboi Carti,JumpOutTheHouse,16171,Playboi Carti:JumpOutTheHouse,NaN,NaN
...,...,...,...,...,...,...,...
24596,2022-01-09 23:30,Juice WRLD,Doom,2623,Juice WRLD:Doom,NaN,NaN
24597,2022-01-09 23:30,Yeat,Gët Busy,42321,Yeat:Gët Busy,NaN,NaN
24598,2022-01-09 23:34,Trippie Redd,Miss The Rage (feat. Playboi Carti),236883,Trippie Redd:Miss The Rage (feat. Playboi Carti),Miss The Rage (feat. Playboi Carti),2BITQ360Knh6qNAOqR7Dyq
24599,2022-01-09 23:36,Playboi Carti,JumpOutTheHouse,93385,Playboi Carti:JumpOutTheHouse,NaN,NaN


In [106]:
#expanding the genres for each index and track uri 
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,alternative r&b
0,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,indie r&b
0,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,uk contemporary r&b
1,4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,melodic rap
1,4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,rap


In [113]:
#convert data frame to csv file
df_genre.to_csv('GenreTable.csv')
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done


In [462]:
df_genre

,track_uri,artist_uri,genres
0,2kMflL8jH2U1UaEhS6W25z,2e0Bkreg0zzfE5umvfKQXf,"[alternative r&b, indie r&b, uk contemporary r&b]"
1,4E08gEca6avoGx5SS4MM6T,5zctI4wO9XSKS8XwcnqEHk,"[melodic rap, rap, rap conscient, trap, vapor ..."
2,6toQdWWc4noiOk3Eo5mVDS,3Y2MLcrD3ZQXRXmtn4fZQx,"[florida rap, viral rap]"
3,4MdraTedPRjTzFhXKPvnfb,6lOaHNmL3l3hKDGNf0egfs,[detroit trap]
4,1S1dOc3yIPNt4sNxhlOghf,6oMuImdp5ZcFhWP0ESe6mG,"[atl hip hop, hip hop, pop rap, rap, trap]"
...,...,...,...
1009,6h4c2iDPVb8GGC6pDnVIIV,6tUJpYN2aYiXbzAcg0pIOo,"[alternative r&b, chill r&b, dance pop, gauze ..."
1010,6J9FgTr3z44Bw6ABeVL415,13ubrt8QOOCPljQ2FL1Kca,"[east coast hip hop, hip hop, rap, trap]"
1011,0qQZ4lxmzitLd6w5WQXrKE,6aLoJJxz7MV2iZ423S8tJC,"[atl hip hop, atl trap, ohio hip hop, pop rap,..."
1012,38iBrrbbXyWaSPkmuDNgjZ,0fA0VVWsXO9YnASrzqfmYu,"[hip hop, ohio hip hop, rap]"


In [464]:
df_genre["genres"]

0       [alternative r&b, indie r&b, uk contemporary r&b]
1       [melodic rap, rap, rap conscient, trap, vapor ...
2                                [florida rap, viral rap]
3                                          [detroit trap]
4              [atl hip hop, hip hop, pop rap, rap, trap]
                              ...                        
1009    [alternative r&b, chill r&b, dance pop, gauze ...
1010             [east coast hip hop, hip hop, rap, trap]
1011    [atl hip hop, atl trap, ohio hip hop, pop rap,...
1012                         [hip hop, ohio hip hop, rap]
1013      [indie pop rap, pop rap, rap, rhode island rap]
Name: genres, Length: 1014, dtype: object